In [128]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
---

## Importamos los datos

In [129]:
df = pd.read_pickle('../../datos/tratados/nonuls_nooutliers_df_employee_data.pkl').reset_index(drop=True)
df.sample(3)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
1253,Low,Medium,Good,50,False,Travel_Frequently,Research & Development,3,Master,Medical,Male,4,Research Scientist,Married,61420,3.00,13,2,28.00,3,8,0,7,Medium,Excellent
1004,High,High,Best,45,False,Non-Travel,Sales,1,Master,Technical Degree,Male,1,Research Scientist,Married,170480,4.00,15,0,10.00,3,8,5,7,Medium,Excellent
789,Low,Very High,Better,37,False,Travel_Rarely,Research & Development,2,College,Medical,Male,1,Research Scientist,Married,20220,1.00,12,0,10.00,1,10,3,7,High,Excellent


---
---
## Encoding

#### Vemos si nuestras variables categóricas tienen orden o no.

In [130]:
df_to_encode = df.copy()

In [131]:
ordered_vars, unordered_vars = sup_encod.detectar_orden_cat(df=df_to_encode, lista_cat=df_to_encode.select_dtypes('O'), var_respuesta='Attrition')

Evaluando variable: ENVIRONMENTSATISFACTION


Attrition,False,True
EnvironmentSatisfaction,,
High,389,63
Low,217,71
Medium,245,42
Very High,384,61
unknown,19,3


La variable EnvironmentSatisfaction TIENE orden. (relación con la VR) ✅ [0.0006]
___________________________________________________________________________ 

Evaluando variable: JOBSATISFACTION


Attrition,False,True
JobSatisfaction,,
High,370,71
Low,214,68
Medium,234,48
Very High,420,52
unknown,16,1


La variable JobSatisfaction TIENE orden. (relación con la VR) ✅ [0.0001]
___________________________________________________________________________ 

Evaluando variable: WORKLIFEBALANCE


...

La variable MaritalStatus TIENE orden. (relación con la VR) ✅ [0.0]
___________________________________________________________________________ 

Evaluando variable: MANAGER_OPINION_JOBINVOLVEMENT


Attrition,False,True
Manager_opinion_JobInvolvement,,
High,751,136
Low,67,19
Medium,313,60
Very High,123,25


La variable Manager_opinion_JobInvolvement NO TIENE orden (sin relación con la VR) ❌ [0.4325]
___________________________________________________________________________ 

Evaluando variable: MANAGER_OPINION_PERFORMANCERATING


Attrition,False,True
Manager_opinion_PerformanceRating,,
Excellent,1071,204
Outstanding,183,36


La variable Manager_opinion_PerformanceRating NO TIENE orden (sin relación con la VR) ❌ [0.9493]
___________________________________________________________________________ 

Las variables ordenadas (relacionadas) son: ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'BusinessTravel', 'Department', 'EducationField', 'MaritalStatus']

Las variables no ordenadas (no relacionadas) son: ['Education', 'Gender', 'JobRole', 'Manager_opinion_JobInvolvement', 'Manager_opinion_PerformanceRating']
___________________________________________________________________________ 



#### Una vez vista la relación de nuestras variables con nuestra variable respuesta, preparamos el diccionario para nuestra clase Encoder.

#### para las que no tienen orden utilizaré el 'One-Hot-Encoding' y para las que si el 'Target' encoding.

In [132]:
encoding_dict = {"onehot": unordered_vars, 
                "dummies": [], 
                'ordinal' : { },
                "label": [],
                "frequency": [],
                "target": ordered_vars
                }

In [133]:
encoder = sup_encod.Encoding(df_to_encode, encoding_dict, variable_respuesta='Attrition')

df_encoded, one_hot_transformer = encoder.one_hot_encoding()
df_encoded, target_transformer = encoder.target_encoding()

df_encoded.sample(3)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Education_Bachelor,Education_Below College,Education_College,Education_Doctor,Education_Master,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,Manager_opinion_JobInvolvement_High,Manager_opinion_JobInvolvement_Low,Manager_opinion_JobInvolvement_Medium,Manager_opinion_JobInvolvement_Very High,Manager_opinion_PerformanceRating_Excellent,Manager_opinion_PerformanceRating_Outstanding
1026,0.15,0.11,0.17,55,False,0.14,0.15,2,0.16,1,0.12,27810,8.00,13,1,19.00,2,5,0,4,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00
612,0.14,0.11,0.17,48,False,0.14,0.15,1,0.15,2,0.26,74060,6.00,14,2,25.00,2,1,0,0,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00
142,0.14,0.24,0.14,28,False,0.14,0.15,7,0.17,3,0.11,119570,1.00,13,0,10.00,2,10,1,8,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00


#### guardamos nuestros encoders

In [ ]:
with open(f'../../transformers/base/one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(one_hot_transformer, f)

with open(f'../../transformers/base/target_encoder.pkl', 'wb') as f:
    pickle.dump(target_transformer, f)

---
---
## Featuring Scaling

#### En este caso voy a utiliazar el RobustScaler debido a que los outlayers que tenemos no son muchos (solo en algunas categorías) y tampoco son errores, por lo que los quiero considerar.

In [134]:
df_to_transform = df_encoded.copy()


#### se que aquí hay valores que podríamos haber categorizado también, pero de momento vamos a probar así y en futuros modelos probaremos categorizándolos.

In [135]:
num_cats = df_to_encode.select_dtypes(include=['number', np.number])

num_cats.sample()

,Age,DistanceFromHome,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
147,42,10,2,28370,2.00,13,1,19.00,0,5,0,2


In [136]:
scaler = sup_fea.FeatureScaling(num_cats, num_cats.columns)

df_num_cats_scaled = scaler.robust_scaler(path='../../transformers/base/', transformer_name='robust_scaler')

df_num_cats_scaled.sample()

,Age,DistanceFromHome,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
704,1.23,-0.17,-0.50,-0.40,2.33,0.67,0.00,0.22,3.00,0.00,-0.33,0.20


#### Unimos los datos escalados con los datos encodeados

In [137]:
df_encoded[num_cats.columns] = df_num_cats_scaled

df_encoded.sample()

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Education_Bachelor,Education_Below College,Education_College,Education_Doctor,Education_Master,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,Manager_opinion_JobInvolvement_High,Manager_opinion_JobInvolvement_Low,Manager_opinion_JobInvolvement_Medium,Manager_opinion_JobInvolvement_Very High,Manager_opinion_PerformanceRating_Excellent,Manager_opinion_PerformanceRating_Outstanding
1309,0.25,0.16,0.14,0.38,False,0.14,0.15,0.00,0.15,1.00,0.11,0.40,0.67,-0.33,0.00,0.33,0.00,0.67,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00


---
---

## Salvamos los datos

In [ ]:
df_encoded.to_pickle('../../datos/tratados/encoded_scaled_df_employee_data.pkl')